<a href="https://colab.research.google.com/github/armandoordonez/eda_couse/blob/main/MICE_imputation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Based on: https://www.machinelearningplus.com/machine-learning/mice-imputation/#google_vignette
# need to enable iterative imputer explicitly since its still experimental

# For understanding imputation please read: https://scikit-learn.org/stable/modules/impute.html

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [2]:
import pandas as pd
file_path = "https://raw.githubusercontent.com/armandoordonez/eda_couse/main/data/Churn_Modelling_m.csv"
df = pd.read_csv(file_path)
print(df.shape)
df.head()

(10000, 14)


,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619.0,France,Female,42.0,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608.0,Spain,Female,41.0,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502.0,France,NaN,NaN,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699.0,France,NaN,39.0,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850.0,Spain,Female,43.0,2,NaN,1,1,1,79084.10,0


In [3]:
# Define imputer
imputer = IterativeImputer(random_state=100, max_iter=10)

In [4]:
# Use Numeric Features
df_train = df.loc[:, ["Balance", "Age", "Exited"]]
df_train.head()

,Balance,Age,Exited
0,0.00,42.0,1
1,83807.86,41.0,0
2,159660.80,NaN,1
3,0.00,39.0,0
4,NaN,43.0,0


In [5]:
# fit on the dataset
imputer.fit(df_train)

IterativeImputer(random_state=100)

In [6]:
df_imputed = imputer.transform(df_train)
df_imputed[:10]

array([[0.00000000e+00, 4.20000000e+01, 1.00000000e+00],
       [8.38078600e+04, 4.10000000e+01, 0.00000000e+00],
       [1.59660800e+05, 4.47681408e+01, 1.00000000e+00],
       [0.00000000e+00, 3.90000000e+01, 0.00000000e+00],
       [7.25930035e+04, 4.30000000e+01, 0.00000000e+00],
       [1.13755780e+05, 4.40000000e+01, 1.00000000e+00],
       [0.00000000e+00, 5.00000000e+01, 0.00000000e+00],
       [1.15046740e+05, 2.90000000e+01, 1.00000000e+00],
       [1.42051070e+05, 4.40000000e+01, 0.00000000e+00],
       [1.34603880e+05, 2.70000000e+01, 0.00000000e+00]])

In [7]:
# Replace with imputed values
df.loc[:, ["Balance", "Age", "Exited"]] = df_imputed
df.head(10)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619.0,France,Female,42.000000,2,0.000000,1,1,1,101348.88,1
1,2,15647311,Hill,608.0,Spain,Female,41.000000,1,83807.860000,1,0,1,112542.58,0
2,3,15619304,Onio,502.0,France,NaN,44.768141,8,159660.800000,3,1,0,113931.57,1
3,4,15701354,Boni,699.0,France,NaN,39.000000,1,0.000000,2,0,0,93826.63,0
4,5,15737888,Mitchell,850.0,Spain,Female,43.000000,2,72593.003473,1,1,1,79084.10,0
5,6,15574012,Chu,645.0,Spain,Male,44.000000,8,113755.780000,2,1,0,NaN,1
6,7,15592531,Bartlett,822.0,France,Male,50.000000,7,0.000000,2,1,1,10062.80,0
7,8,15656148,Obinna,376.0,Germany,Female,29.000000,4,115046.740000,4,1,0,119346.88,1
8,9,15792365,He,501.0,France,Male,44.000000,4,142051.070000,2,0,1,74940.50,0
9,10,15592389,H?,NaN,France,Male,27.000000,2,134603.880000,1,1,1,71725.73,0


In [8]:
import miceforest as mf

In [9]:
# Create kernel.
kds = mf.ImputationKernel(
  df_train,
  save_all_iterations=True,
  random_state=100
)

# Run the MICE algorithm for 2 iterations
kds.mice(2)

# Return the completed dataset.
df_imputed = kds.complete_data()

In [10]:
df_train.head()

,Balance,Age,Exited
0,0.00,42.0,1
1,83807.86,41.0,0
2,159660.80,NaN,1
3,0.00,39.0,0
4,NaN,43.0,0


In [11]:
df_imputed.head()

# It has predicted a value of '55' for the missing record.

# Let's run for 5 more iterations and predict again.

,Balance,Age,Exited
0,0.00,42.0,1
1,83807.86,41.0,0
2,159660.80,55.0,1
3,0.00,39.0,0
4,133214.88,43.0,0


In [12]:
kds.mice(iterations=5, n_estimators=50)

In [13]:
df_imputed2 = kds.complete_data()
df_imputed2.head()

,Balance,Age,Exited
0,0.00,42.0,1
1,83807.86,41.0,0
2,159660.80,39.0,1
3,0.00,39.0,0
4,178074.33,43.0,0


In [ ]:
# The prediction has now changed from 55 to 39.


In [14]:
# Let's compare the actual value by loading the original data that does not contain the missing values

import pandas as pd
file_path = "https://raw.githubusercontent.com/armandoordonez/eda_couse/main/data/Churn_Modelling_orig.csv"
dfs = pd.read_csv(file_path)
dfs[['Balance', 'Age', 'Exited']].head()

,Balance,Age,Exited
0,0.00,42,1
1,83807.86,41,0
2,159660.80,42,1
3,0.00,39,0
4,125510.82,43,0
